In [29]:
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
from scipy.stats import pearsonr as ps

X, y = load_iris(return_X_y = True)

classes = set(y)
X_per_cl = {cl: np.array([it
                    for num, it in enumerate(X)
                    if y[num] == cl]) for cl in classes}
means = dict(map(lambda kv: (kv[0], kv[1].mean(axis = 0)), iter(X_per_cl.items())))
deltas = {'{}-{}'.format(class_1, class_2): means[class_1] - means[class_2] 
        for class_1 in classes
        for class_2 in classes
        if class_1 != class_2}
pd.DataFrame(deltas)


,0-1,0-2,1-0,1-2,2-0,2-1
0,-0.930,-1.582,0.930,-0.652,1.582,0.652
1,0.648,0.444,-0.648,-0.204,-0.444,0.204
2,-2.796,-4.088,2.796,-1.292,4.088,1.292
3,-1.082,-1.782,1.082,-0.700,1.782,0.700


In [30]:
print('Effects size:')
for num, x in enumerate(X.T):
    print('Param {}: {}'.format(num, ps(x, y)[0]))

Effects size:
Param 0: 0.7825612318100814
Param 1: -0.4194462002600275
Param 2: 0.9490425448523336
Param 3: 0.9564638238016173
